In [2]:
import pandas as pd
import re
import os
from IPython.display import clear_output
import ToolBox.kit_function as kit

In [3]:
listSheet = os.listdir('Dados')
table_master = pd.DataFrame({})
columns = ['Nome','codigo','prc','imp','link']

for index,sheet in enumerate(listSheet):
    clear_output(wait=False)  
    print(f'Concatenando {sheet} na Tabela Master.\n Planilha {index+1} de {len(listSheet)}')
    aux_sheet = kit.readExcel(str('Dados/' + sheet) , columns)
    table_master = pd.concat([table_master, aux_sheet])

table_master =  table_master.reset_index(drop=True)

Concatenando VIRA MACHO.xlsx na Tabela Master.
 Planilha 127 de 127


In [45]:
def regexClean(string,regexExpression):
    result = re.compile(regexExpression).findall(string)
    result = (result[0] or result[1])
    
    if isinstance(result, tuple):
        return result[0] or result[1]
    else:
        return result


In [56]:
expCodigo = r"[0-9]+" #5400455
expPrc = r"[0-9]+[,][0-9]+" #10,47

expIPI1 = r'[IP]+[: ]+[0-9]+%' #IPI: 5%
expIPI2 = r'[IP]+[: ]+[0-9]+[,][0-9]+%' #IPI: 5,5%

expST1 = r'[ST]+[: ]+[0-9]+%' #ST: 27%
expST2 = r'[ST]+[: ]+[0-9]+[,][0-9]+%' #ST: 27,68%

expPOR1 = r'[0-9]+%'
expPOR2 = r'[0-9]+[,][0-9]+%'

# Combina ambas as expressões
combined_ST = f"({expST1})|({expST2})"
combined_IPI = f"({expIPI1})|({expIPI2})"
combined_POR = f"({expPOR1})|({expPOR2})"

df = {'Nome':[],
    'codigo': [],
    'prc': [],
    'ipi': [],
    'st': [],
    'link':[]}

In [58]:
codigo = table_master['codigo'][0]
preco = table_master['prc'][0]
ipi = table_master['imp'][0]
st = table_master['imp'][0]

codigo = regexClean(codigo,expCodigo)
preco = regexClean(preco,expPrc)
ipi = regexClean(regexClean(ipi,combined_IPI),combined_POR)
st = regexClean(regexClean(st,combined_ST),combined_POR)

print(f'{codigo}\n{preco}\n{ipi}\n{st}')

Cód. 5400455 • UN
R$ 2,71info_outline
+ IPI: 5% + ST: 27,68%
+ IPI: 5% + ST: 27,68%
5400455
2,71
5%
27,68%


In [5]:
for line in range(len(table_master)):
    codigo = iter(regexClean(table_master['codigo'][line],expCodigo))
    preco = iter(regexClean(table_master['prc'][line],expPrc))
    ipi = (match[0] or match[1] for match in regexClean(table_master['imp'][line],combined_IPI))
    st = (match[0] or match[1] for match in regexClean(table_master['imp'][line],combined_ST))

    #Captura a String Válida
    codigo = next(codigo,'')
    preco = next(preco,'')
    ipi = next(ipi,'')    
    st = next(st,'')    
    
    df['Nome'].append(table_master['Nome'][line])
    df['codigo'].append(codigo)
    df['prc'].append(preco)
    df['ipi'].append(ipi)
    df['st'].append(st)
    df['link'].append(table_master['link'][line])
    

In [14]:
xlsx_dir = 'Links/'
titleCategoria = 'Scrap-mtx'
mtxTable = pd.DataFrame(df)
file_name = xlsx_dir + titleCategoria + '.xlsx'
mtxTable.to_excel(file_name, index=False)
print(f'A planilha {titleCategoria}.xlsx foi salva em {file_name}')

A planilha Scrap-mtx.xlsx foi salva em Links/Scrap-mtx.xlsx
